<a href="https://colab.research.google.com/github/Rajora0/nlp_studies/blob/main/Word2Vec_treinamento_de_Word_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word2Vec: treinamento de Word Embedding

## Montar Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive/')

!ls '/gdrive/MyDrive/NLP_ESTUDOS/Dados/word2vec'

Drive already mounted at /gdrive/; to attempt to forcibly remount, call drive.mount("/gdrive/", force_remount=True).
cbow_s300.txt  skip_s300.txt  teste.csv  treino.csv


## Carregando os dados

In [ ]:
import numpy as np
import pandas as pd
import spacy
import warnings
warnings.filterwarnings("ignore")

In [ ]:
path = '/gdrive/MyDrive/NLP_ESTUDOS/Dados/word2vec/'

In [ ]:
dados_treino = pd.read_csv(path+"treino.csv")
dados_teste = pd.read_csv(path+"teste.csv")

In [ ]:
dados_treino.head()

,title,text,date,category,subcategory,link
0,"Após polêmica, Marine Le Pen diz que abomina n...",A candidata da direita nacionalista à Presidên...,2017-04-28,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
1,"Macron e Le Pen vão ao 2º turno na França, em ...",O centrista independente Emmanuel Macron e a d...,2017-04-23,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
2,"Apesar de larga vitória nas legislativas, Macr...",As eleições legislativas deste domingo (19) na...,2017-06-19,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/06/189...
3,"Governo antecipa balanço, e Alckmin anuncia qu...",O número de ocorrências de homicídios dolosos ...,2015-07-24,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/07...
4,"Após queda em maio, a atividade econômica sobe...","A economia cresceu 0,25% no segundo trimestre,...",2017-08-17,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/08/1...


In [ ]:
dados_teste.head()

,title,text,date,category,subcategory,link
0,Grandes irmãos,"RIO DE JANEIRO - O Brasil, cada vez menos famí...",2017-03-06,colunas,ruycastro,http://www1.folha.uol.com.br/colunas/ruycastro...
1,Haddad congela orçamento e suspende emendas de...,"O prefeito de São Paulo, Fernando Haddad (PT),...",2016-08-10,colunas,monicabergamo,http://www1.folha.uol.com.br/colunas/monicaber...
2,Proposta de reforma da Fifa tem a divulgação d...,"A Fifa divulgou, nesta quinta (10), um relatór...",2015-10-09,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/09/1...
3,"Mercado incipiente, internet das coisas conect...","Bueiros, coleiras, aparelhos hospitalares, ele...",2016-11-09,mercado,NaN,http://www1.folha.uol.com.br/mercado/2016/09/1...
4,"Mortes: Psicanalista, estudou o autismo em cri...",Toda vez que o grupo de amigos de Silvana Rabe...,2017-02-07,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2017/07...


## Spacy

- https://spacy.io/

In [ ]:
#!pip install spacy -qq
#!python -m spacy download pt_core_news_sm -qq

In [ ]:
nlp = spacy.load("pt_core_news_sm")
nlp

In [ ]:
texto = "Rio de Janeiro é uma cidade maravilhosa"
doc = nlp(texto)

In [ ]:
type(doc)

spacy.tokens.doc.Doc

In [ ]:
doc[0]

Rio

In [ ]:
doc.ents

(Rio de Janeiro,)

In [ ]:
doc[0].is_stop

False

In [ ]:
doc[1].is_stop

True

## Tratando nossos dados

In [ ]:
from time import time

In [ ]:
textos_para_tratamento = (titulos.lower() for titulos in dados_treino["title"])


In [ ]:
def trata_textos(doc):
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text)

    if len(tokens_validos) > 2:
        return  " ".join(tokens_validos)

texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
doc = nlp(texto)
trata_textos(doc)

'Rio Janeiro cidade maravilhosa'

In [ ]:
texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
doc = nlp(texto)
trata_textos(doc)

'Rio Janeiro cidade maravilhosa'

In [ ]:
t0 = time()
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                        batch_size = 1000,
                                                        n_process = -1)]

tf = time() - t0

print(tf/60)

3.615842342376709


In [ ]:
titulos_tratados = pd.DataFrame({"titulo": textos_tratados})
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron e le pen a o turno frança revés siglas ...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço e alckmin anuncia que...
4,queda maio a atividade econômica sobe junho bc


## Construindo o Word2Vec

In [ ]:
import logging
from gensim.models import Word2Vec

In [ ]:
print(len(titulos_tratados))
titulos_tratados = titulos_tratados.dropna().drop_duplicates()
print(len(titulos_tratados))

90000
86113


In [ ]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]

In [ ]:
logging.basicConfig(format="%(asctime)s : - %(message)s", level = logging.INFO)

w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo.build_vocab(lista_lista_tokens, progress_per=5000)

2022-05-12 12:00:45,402 : - collecting all words and their counts
2022-05-12 12:00:45,404 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-05-12 12:00:45,426 : - PROGRESS: at sentence #5000, processed 34716 words, keeping 10129 word types
2022-05-12 12:00:45,443 : - PROGRESS: at sentence #10000, processed 69298 words, keeping 14909 word types
2022-05-12 12:00:45,458 : - PROGRESS: at sentence #15000, processed 103841 words, keeping 18223 word types
2022-05-12 12:00:45,475 : - PROGRESS: at sentence #20000, processed 138620 words, keeping 20969 word types
2022-05-12 12:00:45,492 : - PROGRESS: at sentence #25000, processed 173257 words, keeping 23410 word types
2022-05-12 12:00:45,516 : - PROGRESS: at sentence #30000, processed 207976 words, keeping 25453 word types
2022-05-12 12:00:45,537 : - PROGRESS: at sentence #35000, processed 242567 words, keeping 27263 word types
2022-05-12 12:00:45,554 : - PROGRESS: at sentence #40000, processed 277254 words, keeping 2899

## Treinando o Word2Vec

In [ ]:
dir(w2v_modelo)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_input_data_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_job_params',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_minimize_model',
 '_raw_word_count',
 '_save_specials',
 '_set_train_params',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_update_job_params',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'accuracy',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'ca

In [ ]:
w2v_modelo.corpus_count

86113

In [ ]:
w2v_modelo.train(lista_lista_tokens, 
                 total_examples=w2v_modelo.corpus_count,
                 epochs = 30)

2022-05-12 12:00:48,672 : - training model with 3 workers on 13006 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=2
2022-05-12 12:00:49,697 : - EPOCH 1 - PROGRESS: at 65.27% examples, 324909 words/s, in_qsize 5, out_qsize 0
2022-05-12 12:00:50,149 : - worker thread finished; awaiting finish of 2 more threads
2022-05-12 12:00:50,151 : - worker thread finished; awaiting finish of 1 more threads
2022-05-12 12:00:50,176 : - worker thread finished; awaiting finish of 0 more threads
2022-05-12 12:00:50,178 : - EPOCH - 1 : training on 597929 raw words (502873 effective words) took 1.5s, 337572 effective words/s
2022-05-12 12:00:51,235 : - EPOCH 2 - PROGRESS: at 68.60% examples, 330871 words/s, in_qsize 5, out_qsize 0
2022-05-12 12:00:51,623 : - worker thread finished; awaiting finish of 2 more threads
2022-05-12 12:00:51,631 : - worker thread finished; awaiting finish of 1 more threads
2022-05-12 12:00:51,641 : - worker thread finished; awaiting finish of 0 more t

(15086840, 17937870)

In [ ]:
w2v_modelo.wv.most_similar("google")

2022-05-12 12:01:31,584 : - precomputing L2-norms of word weight vectors


[('apple', 0.5999081134796143),
 ('facebook', 0.5410079956054688),
 ('amazon', 0.5001060962677002),
 ('waze', 0.4755364656448364),
 ('fbi', 0.47435659170150757),
 ('uber', 0.47419095039367676),
 ('airbnb', 0.4717792868614197),
 ('web', 0.4630432724952698),
 ('buffett', 0.44291895627975464),
 ('canais', 0.43591761589050293)]

In [ ]:
w2v_modelo.wv.most_similar("microsoft")

[('unilever', 0.6132704019546509),
 ('sony', 0.5909816026687622),
 ('amazon', 0.5897475481033325),
 ('tesla', 0.588370680809021),
 ('canais', 0.5589038133621216),
 ('walmart', 0.5515804290771484),
 ('sky', 0.5429564714431763),
 ('lego', 0.5395606756210327),
 ('chrysler', 0.5393645763397217),
 ('street', 0.5308616757392883)]

In [ ]:
w2v_modelo.wv.most_similar("barcelona")

[('barça', 0.6314722299575806),
 ('chelsea', 0.6138635873794556),
 ('psg', 0.6048184633255005),
 ('bayern', 0.5881408452987671),
 ('leicester', 0.5868918895721436),
 ('juventus', 0.5865355134010315),
 ('munique', 0.5690410137176514),
 ('figueirense', 0.5634949207305908),
 ('madrid', 0.5630789995193481),
 ('united', 0.5603851079940796)]

In [ ]:
w2v_modelo.wv.most_similar("messi")

[('suárez', 0.6128507256507874),
 ('cristiano', 0.5684494972229004),
 ('cavani', 0.5360791683197021),
 ('neymar', 0.5292573571205139),
 ('benzema', 0.5154162049293518),
 ('tevez', 0.5143914222717285),
 ('barcelona', 0.510454535484314),
 ('barça', 0.5050179958343506),
 ('psg', 0.49441391229629517),
 ('chuteiras', 0.4937223196029663)]

In [ ]:
w2v_modelo.wv.most_similar("gm")

[('chrysler', 0.6866105198860168),
 ('honda', 0.6504881381988525),
 ('fiat', 0.6390398144721985),
 ('toyota', 0.6367384195327759),
 ('volks', 0.6337071657180786),
 ('embraer', 0.6161869764328003),
 ('renault', 0.6044415235519409),
 ('tesla', 0.5988296270370483),
 ('sabmiller', 0.5901549458503723),
 ('inbev', 0.5830637812614441)]

In [ ]:
#Treinamento do modelo Skip-Gram
w2v_modelo_sg = Word2Vec(sg = 1,
                      window = 5,
                      size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo_sg.build_vocab(lista_lista_tokens, progress_per=5000)

w2v_modelo_sg.train(lista_lista_tokens, 
                 total_examples=w2v_modelo_sg.corpus_count,
                 epochs = 30)

2022-05-12 12:01:31,713 : - collecting all words and their counts
2022-05-12 12:01:31,716 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-05-12 12:01:31,739 : - PROGRESS: at sentence #5000, processed 34716 words, keeping 10129 word types
2022-05-12 12:01:31,765 : - PROGRESS: at sentence #10000, processed 69298 words, keeping 14909 word types
2022-05-12 12:01:31,787 : - PROGRESS: at sentence #15000, processed 103841 words, keeping 18223 word types
2022-05-12 12:01:31,808 : - PROGRESS: at sentence #20000, processed 138620 words, keeping 20969 word types
2022-05-12 12:01:31,825 : - PROGRESS: at sentence #25000, processed 173257 words, keeping 23410 word types
2022-05-12 12:01:31,839 : - PROGRESS: at sentence #30000, processed 207976 words, keeping 25453 word types
2022-05-12 12:01:31,852 : - PROGRESS: at sentence #35000, processed 242567 words, keeping 27263 word types
2022-05-12 12:01:31,868 : - PROGRESS: at sentence #40000, processed 277254 words, keeping 2899

(15086933, 17937870)

In [ ]:
w2v_modelo_sg.wv.most_similar("google")

2022-05-12 12:04:10,703 : - precomputing L2-norms of word weight vectors


[('reguladores', 0.43487855792045593),
 ('waze', 0.424803227186203),
 ('yahoo', 0.3967300355434418),
 ('android', 0.3943891227245331),
 ('apple', 0.39112749695777893),
 ('patentes', 0.37960922718048096),
 ('concorda', 0.3726450800895691),
 ('facebook', 0.3719591200351715),
 ('buffett', 0.3700791895389557),
 ('verizon', 0.36772656440734863)]

In [ ]:
w2v_modelo.wv.most_similar("google")

[('apple', 0.5999081134796143),
 ('facebook', 0.5410079956054688),
 ('amazon', 0.5001060962677002),
 ('waze', 0.4755364656448364),
 ('fbi', 0.47435659170150757),
 ('uber', 0.47419095039367676),
 ('airbnb', 0.4717792868614197),
 ('web', 0.4630432724952698),
 ('buffett', 0.44291895627975464),
 ('canais', 0.43591761589050293)]

In [ ]:
w2v_modelo_sg.wv.most_similar("gm")

[('metalúrgicos', 0.5848363637924194),
 ('honda', 0.5092787742614746),
 ('motors', 0.5050153732299805),
 ('airbags', 0.49535268545150757),
 ('bmw', 0.47285881638526917),
 ('audi', 0.4693719148635864),
 ('cubatão', 0.46607792377471924),
 ('airbag', 0.46572691202163696),
 ('montadora', 0.4585378170013428),
 ('patente', 0.45767301321029663)]

In [ ]:
w2v_modelo.wv.most_similar("gm")

[('chrysler', 0.6866105198860168),
 ('honda', 0.6504881381988525),
 ('fiat', 0.6390398144721985),
 ('toyota', 0.6367384195327759),
 ('volks', 0.6337071657180786),
 ('embraer', 0.6161869764328003),
 ('renault', 0.6044415235519409),
 ('tesla', 0.5988296270370483),
 ('sabmiller', 0.5901549458503723),
 ('inbev', 0.5830637812614441)]

In [ ]:
w2v_modelo.wv.save_word2vec_format(path+"modelo_cbow.txt", binary=False)
w2v_modelo_sg.wv.save_word2vec_format(path+"modelo_skipgram.txt", binary=False)

2022-05-12 12:04:20,343 : - storing 13006x300 projection weights into /gdrive/MyDrive/NLP_ESTUDOS/Dados/word2vec/modelo_cbow.txt
2022-05-12 12:04:23,109 : - storing 13006x300 projection weights into /gdrive/MyDrive/NLP_ESTUDOS/Dados/word2vec/modelo_skipgram.txt
